In [1]:
import pandas as pd
import numpy as np

ny_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_newyork.csv"
austin_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_austin.csv"
cali_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_california.csv"

ny_data = pd.read_csv(ny_path)
austin_data = pd.read_csv(austin_path)
cali_data = pd.read_csv(cali_path)

ny_user_ids = ny_data.dataid.unique().tolist()
austin_user_ids = austin_data.dataid.unique().tolist()
cali_user_ids = cali_data.dataid.unique().tolist()

In [2]:
cali_user_ids + austin_user_ids + ny_user_ids

[3687,
 6377,
 7062,
 8574,
 9213,
 203,
 1450,
 1524,
 2606,
 3864,
 7114,
 1731,
 4495,
 8342,
 3938,
 5938,
 8061,
 9775,
 4934,
 8733,
 9612,
 9836,
 6547,
 661,
 1642,
 2335,
 2361,
 2818,
 3039,
 3456,
 3538,
 4031,
 4373,
 4767,
 5746,
 6139,
 7536,
 7719,
 7800,
 7901,
 7951,
 8156,
 8386,
 8565,
 9019,
 9160,
 9922,
 9278,
 4550,
 558,
 2358,
 3700,
 1417,
 5679,
 5058,
 2318,
 5997,
 950,
 5982,
 5587,
 1222,
 387,
 3000,
 4283,
 3488,
 3517,
 9053,
 3996,
 27,
 142,
 914,
 2096,
 1240]

In [ ]:
def plot_range_with_syn_values(df, colname, syn_df, month, weekday):
    # Filter the DataFrame for the given month and weekday
    filtered_df = df[(df['month'] == month) & (df['weekday'] == weekday)]
    
    # Ensure there is data for the given month and weekday
    if filtered_df.empty:
        print(f"No data for month={month}, weekday={weekday}")
        return
    
    # Convert the 'grid' column to a numpy array
    array_data = np.array(filtered_df[colname].to_list())

    # Calculate min and max values across all rows
    min_values = np.min(array_data, axis=0)
    max_values = np.max(array_data, axis=0)

    # Ensure the synthetic data DataFrame has the same structure and filter it
    syn_filtered_df = syn_df[(syn_df['month'] == month) & (syn_df['weekday'] == weekday)]

    # Ensure there is synthetic data for the given month and weekday
    if syn_filtered_df.empty:
        print(f"No synthetic data for month={month}, weekday={weekday}")
        return

    syn_values = np.array(syn_filtered_df['grid'].to_list()).squeeze()

    # Generate timestamps for the x-axis
    timestamps = pd.date_range(start='00:00', end='23:45', freq='15T').strftime('%H:%M')

    plt.figure(figsize=(15, 7))
    plt.fill_between(timestamps, min_values, max_values, color='gray', alpha=0.5, label='Range of values')
    
    # Plot the synthetic time series
    plt.plot(timestamps, syn_values[2], color='blue', marker='o', markersize=2, linestyle='-', label='Synthetic values')

    plt.title(f'Range of Values and Synthetic Data Comparison for Month={month}, Weekday={weekday}')
    plt.xlabel('Time of Day')
    plt.ylabel('Values')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_range_with_syn_values(unnormalized_ori, 'grid', unnormalized_syn, 10, 4)

In [ ]:
from sklearn.metrics import mean_squared_error
from data_utils.dataset import PecanStreetDataset, split_dataset
import pandas as pd
import numpy as np

def check_inverse_transform(normalized_dataset, unnormalized_dataset):
    normalized_df = normalized_dataset.data
    unnormalized_df = unnormalized_dataset.data
    transformed = normalized_dataset.inverse_transform(normalized_df)

    mse_list = []

    for idx in range(len(unnormalized_df)):
        unnormalized_timeseries = unnormalized_df.iloc[idx]["timeseries"]
        transformed_timeseries = transformed.iloc[idx]["timeseries"]

        # Ensure the arrays have the same shape
        assert unnormalized_timeseries.shape == transformed_timeseries.shape, "Shape mismatch between transformed and unnormalized timeseries"

        # Calculate the MSE for this row
        mse = mean_squared_error(unnormalized_timeseries, transformed_timeseries)
        mse_list.append(mse)

    # Calculate the average MSE over all rows
    avg_mse = np.mean(mse_list)

    print(f"Average MSE over all rows: {avg_mse}")
    return avg_mse

check_inverse_transform(661)

In [ ]:

USER_IDs = [27,  142,  203,  387,  558,  661,  914,  950, 1222, 1240, 1417,
       1450, 1524, 1642, 1731, 2096, 2318, 2335, 2358, 2361, 2606, 2818,
       3000, 3039, 3456, 3488, 3517, 3538, 3687, 3700, 3864, 3938, 3996,
       4031, 4283, 4373, 4495, 4550, 4767, 4934, 5058, 5587, 5679, 5746,
       5938, 5982, 5997, 6139, 6377, 6547, 7062, 7114, 7536, 7719, 7800,
       7901, 7951, 8061, 8156, 8342, 8386, 8565, 8574, 8733, 9019, 9053,
       9160, 9213, 9278, 9612, 9775, 9836, 9922]


In [1]:
from generator.llm.llm import HF
from generator.llm.preprocessing import Signal2String

model = HF()

/home/fuest/EnData/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.32s/it]


In [2]:
from data_utils.dataset import PecanStreetDataset

data = PecanStreetDataset(normalize=True, include_generation=False)
user_data = data.create_user_dataset(661)

In [9]:
converter = Signal2String(decimal=4)
text = converter.transform(user_data.data.timeseries.iloc[0])

In [14]:
text = text.replace('[', '').replace("]", "")

In [15]:
model.generate(text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ValueError: The model vocabulary size is 128256, but the following tokens were being biased: [128256]

In [16]:
text

'9396,9999,8915,6912,6774,6214,5821,6016,5857,5673,5714,5891,5714,6218,7777,5936,5334,5301,5332,5341,4924,4887,4853,4935,4809,4770,4760,4932,4886,4781,4755,4819,4590,4712,4673,4711,4604,4766,4666,4837,4596,4780,4588,4844,4714,4728,4676,4736,4637,4896,4655,4803,4645,4868,4783,4781,3938,3597,3506,2740,2289,1785,1763,1617,1189,907,827,808,599,426,301,408,308,357,215,337,257,376,473,586,580,715,1000,1270,1265,1528,1971,2626,2768,3007,3340,3850,4041,4266,4841,5139'